# IDR Movie Downloader
### Downloads movies from a well/plate/screen from an idr dataset

#### Import libraries

In [29]:
from idr import connection
import time
from pathos.multiprocessing import ProcessPool
from itertools import repeat
import os
import pandas as pd
import concurrent.futures

#### Functions used for downloading screens/plates and saving annotations/images

In [30]:
#download screen given the screen ID and path, if start is given, download images including and after this plate
def download_screen(screen_num, path, start_plate=0):
    screen = conn.getObject("Screen", screen_num)
    #download all plates in screen
    for plate in screen.listChildren():
        if plate.getId() < start_plate:
            continue
        else:
            download_plate(plate.getId(), path)

def download_plate(plate_num, path, ):
    plate = conn.getObject("Plate", plate_num)
    plate_name = plate.getName()
    print("Downloading " + plate_name)
    
    #make directory for the plate
    os.makedirs(path + plate_name, exist_ok=True)
    
    #iterate through wells in plate and download that well
    donwload_path = path + plate_name + "/"
    well_grid = plate.getWellGrid()
    for row in range(len(well_grid)):
        for col in range(len(well_grid[row])):
            download_well(row, col, well_grid[row][col], donwload_path)
    

def download_well(row, col, well, path):
    #create well folder and save annotations/images to that folder
    well_pos = getWellPosFast(row, col)
    print("Downloading well " + well_pos + ", Total Time: " + str(time.time()-start))
    well_dir = path + well_pos + "/"
    try:
        os.makedirs(well_dir)
        save_annotations(well, well_dir)
        save_movie(well, well_dir)
    except FileExistsError:
        print("Well already downloaded!")
      
#faster implementation of well.getWellPos() that derives pos from row and col  
def getWellPosFast(row, col):
    letter = chr(ord('@')+(row+1))
    num = col+1
    return letter+str(num)
    

def save_annotations(well, path):
    annotations_list = well.listAnnotations()
    
    #annotations are in bad format so have to recreate list before converting to pandas dataframe
    new_annotations_list = []
    for annotation in annotations_list:
        for value in annotation.getValue():
            new_annotations_list.append(value)
    
    pd.DataFrame(new_annotations_list).to_csv(path + "annotations")


def save_movie(well, path):
    image = well.getImage()
    #threaded save of each frame
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(save_frame, repeat(image), range(image.getSizeT()), repeat(path))
    
            
def save_frame(image, t, path):
    rendered_image = image.renderImage(0, t)
    string_t = "{:02d}".format(t) #format 2 as 02 and 19 as 19 for numberical ordering of string filenames
    rendered_image.save(path + "/frame_" + string_t +  ".webp")

#### Information for 1 plate download:
4550 seconds - 76 minutes,
384 wells,
625 mb

#### Information for all plates
510 plates,
644 hours,
318 gb




In [34]:
conn = connection('idr.openmicroscopy.org', 'public', 'public') 
start = time.time()

# download_plate(3454, "idr_downloads/")
#download_screen(1101, "idr_downloads/", start_plate=3457)

plate = conn.getObject("Plate", 3457)
plate_name = plate.getName()
print("Downloading " + plate_name)

path = "idr_downloads/"
#make directory for the plate
os.makedirs(path + plate_name, exist_ok=True)

#iterate through wells in plate and download that well
donwload_path = path + plate_name + "/"
well_grid = plate.getWellGrid()
for row in range(len(well_grid)):
    for col in range(len(well_grid[row])):
        if getWellPosFast(row, col)=="M6":
            download_well(row, col, well_grid[row][col], donwload_path)

Connected to IDR ...


{
    slice = 0
    x = 0
    y = 0
    z = 0
    t = 4
    region = <nil>
    stride = 0
}, <ServiceOptsDict: {'omero.client.uuid': 'ebe5ee58-57d0-4089-9469-01f50691b7b3', 'omero.session.uuid': '39a37002-6fcd-4f87-8a44-aaa8cf931b3d'}>), {})
Traceback (most recent call last):
  File "/home/roshankern/anaconda3/envs/phenotypic_profiling/lib/python3.9/site-packages/omero/gateway/__init__.py", line 4853, in __call__
    return self.f(*args, **kwargs)
  File "/home/roshankern/anaconda3/envs/phenotypic_profiling/lib/python3.9/site-packages/omero_api_RenderingEngine_ice.py", line 913, in renderCompressed
    return _M_omero.api.RenderingEngine._op_renderCompressed.invoke(self, ((_def, ), _ctx))
omero.ResourceError: exception ::omero::ResourceError
{
    serverStackTrace = ome.conditions.ResourceError: Can't create cache file!
	at ome.services.RenderingBean.renderCompressed(RenderingBean.java:559)
	at jdk.internal.reflect.GeneratedMethodAccessor1128.invoke(Unknown Source)
	at java.base/jdk.in

KeyboardInterrupt: 

{
    slice = 0
    x = 0
    y = 0
    z = 0
    t = 26
    region = <nil>
    stride = 0
}, <ServiceOptsDict: {'omero.client.uuid': 'ebe5ee58-57d0-4089-9469-01f50691b7b3', 'omero.session.uuid': '39a37002-6fcd-4f87-8a44-aaa8cf931b3d'}>), {})
Traceback (most recent call last):
  File "/home/roshankern/anaconda3/envs/phenotypic_profiling/lib/python3.9/site-packages/omero/gateway/__init__.py", line 4853, in __call__
    return self.f(*args, **kwargs)
  File "/home/roshankern/anaconda3/envs/phenotypic_profiling/lib/python3.9/site-packages/omero_api_RenderingEngine_ice.py", line 913, in renderCompressed
    return _M_omero.api.RenderingEngine._op_renderCompressed.invoke(self, ((_def, ), _ctx))
omero.ResourceError: exception ::omero::ResourceError
{
    serverStackTrace = ome.conditions.ResourceError: Can't create cache file!
	at ome.services.RenderingBean.renderCompressed(RenderingBean.java:559)
	at jdk.internal.reflect.GeneratedMethodAccessor1128.invoke(Unknown Source)
	at java.base/jdk.i